In [91]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#%matplotlib notebook
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
data = pd.read_excel(r"D:\user interface\final_data_first_outlier_then_4log.xlsx")
data.drop(["Unnamed: 0"], axis=1,inplace=True)

X_train = data.drop(['Category','Sex','CREA'],axis=1)
y_train = data['Category']
# Save the column names
column_names = X_train.columns.tolist()

# Standardize the features
ss = StandardScaler()
X_train = ss.fit_transform(X_train)

# Create an instance of the SMOTE class
smote = SMOTE()

# Generate X_resampled and y_resampled
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Selecting a specified number of samples for each category for testing
num_samples_per_category = 100  # Specify the desired number of samples for each category

test_indices = []
for category in np.unique(y_resampled):
    category_indices = np.where(y_resampled == category)[0]
    category_indices = category_indices[:num_samples_per_category]
    test_indices.extend(category_indices)

# Resetting the index of X_resampled and y_resampled
X_resampled = pd.DataFrame(X_resampled, columns=column_names)
y_resampled = pd.Series(y_resampled)

# Creating a new set of indices for test_indices
new_test_indices = []
for index in test_indices:
    new_test_indices.append(X_resampled.index[index])

# Extracting the corresponding samples for testing
X_test = X_resampled.loc[new_test_indices]
y_test = y_resampled[new_test_indices]

# Dropping the rows for testing samples from X_resampled and y_resampled
X_train = X_resampled.drop(index=new_test_indices)
y_train = y_resampled.drop(index=new_test_indices)


# building the stack lr

In [92]:
from sklearn.svm import SVC
svm = SVC()

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
xg = XGBClassifier()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
rf.fit(X_train, y_train)
svm.fit(X_train, y_train)

xg.fit(X_train, y_train)
dt.fit(X_train, y_train)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression



In [93]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

# Your previous code for defining base classifiers

# Create the StackingClassifier
estimator_list = [
    ('rf', rf),
    ('svm', svm),
    ('xg', xg),
    ('dt', dt)
]

stack_model = StackingClassifier(
    estimators=estimator_list,
    final_estimator=LogisticRegression(max_iter=100000)
)

# Define hyperparameter grid for the final estimator (Logistic Regression)
final_estimator_params = {
    'final_estimator__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'final_estimator__C': np.logspace(-4, 4, 20),
    'final_estimator__solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    'final_estimator__max_iter': [50, 100, 250, 500]
}

# Perform RandomizedSearchCV for the final estimator
final_estimator_search = GridSearchCV(
    stack_model,
    final_estimator_params,
    cv=7,
    return_train_score=False,
    n_jobs=-1
)

final_estimator_search.fit(X_train, y_train)

# Print the results for the final estimator
print({
    'model': 'logistic_regression',
    'best_score': final_estimator_search.best_score_,
    'best_params': final_estimator_search.best_params_
})



"\n# Define hyperparameter grid for the final estimator (Logistic Regression)\nfinal_estimator_params = {\n    'final_estimator__penalty': ['l1', 'l2', 'elasticnet', 'none'],\n    'final_estimator__C': np.logspace(-4, 4, 20),\n    'final_estimator__solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],\n    'final_estimator__max_iter': [50, 100, 250, 500]\n}\n\n# Perform RandomizedSearchCV for the final estimator\nfinal_estimator_search = RandomizedSearchCV(\n    stack_model,\n    final_estimator_params,\n    cv=7,\n    return_train_score=False,\n    n_jobs=-1\n)\n\nfinal_estimator_search.fit(X_train, y_train)\n\n# Print the results for the final estimator\nprint({\n    'model': 'logistic_regression',\n    'best_score': final_estimator_search.best_score_,\n    'best_params': final_estimator_search.best_params_\n})\n\n"

# find accuracy of test data

In [94]:
stack_model.fit(X_train, y_train)
stack_pred = stack_model.predict(X_test)
y_pred=stack_model.predict(X_train)

print(f"stack model train Accuracy : {accuracy_score(y_train, y_pred)}")
from sklearn.metrics import precision_score, recall_score

precision_micro = precision_score(y_test, stack_pred, average='micro')
recall_micro = recall_score(y_test, stack_pred, average='micro')

precision_macro = precision_score(y_test, stack_pred, average='macro')
recall_macro = recall_score(y_test, stack_pred, average='macro')
print(f"test Accuracy : {accuracy_score(y_test, stack_pred)}")
print(f"Micro-average Precision: {precision_micro}")
print(f"Micro-average Recall: {recall_micro}")
print(f"Macro-average Precision: {precision_macro}")
print(f"Macro-average Recall: {recall_macro}")
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, stack_pred)
rmse = mean_squared_error(y_test,stack_pred, squared=False)  # squared=False gives RMSE
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

stack model train Accuracy : 1.0
test Accuracy : 1.0
Micro-average Precision: 1.0
Micro-average Recall: 1.0
Macro-average Precision: 1.0
Macro-average Recall: 1.0
Mean Absolute Error (MAE): 0.0
Root Mean Squared Error (RMSE): 0.0


# For hyperparameter tuning parameters

In [95]:
solver = 'sag'
penalty = 'none'
max_iterations = 500
c_value = 0.0001
stack_model2 = StackingClassifier(
    estimators=estimator_list,
    final_estimator=LogisticRegression(solver=solver, penalty=penalty, max_iter=max_iterations, C=c_value)

)
stack_model2.fit(X_train, y_train)
stack_pred2 = stack_model2.predict(X_test)
print(f"random forest test Accuracy : {accuracy_score(y_test, stack_pred2)}")
from sklearn.metrics import precision_score, recall_score

precision_micro = precision_score(y_test, stack_pred2, average='micro')
recall_micro = recall_score(y_test, stack_pred2, average='micro')

precision_macro = precision_score(y_test, stack_pred2, average='macro')
recall_macro = recall_score(y_test, stack_pred2, average='macro')
print(f"test Accuracy : {accuracy_score(y_test, stack_pred2)}")
print(f"Micro-average Precision: {precision_micro}")
print(f"Micro-average Recall: {recall_micro}")
print(f"Macro-average Precision: {precision_macro}")
print(f"Macro-average Recall: {recall_macro}")
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, stack_pred2)
rmse = mean_squared_error(y_test,stack_pred2, squared=False)  # squared=False gives RMSE
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

random forest test Accuracy : 0.998
test Accuracy : 0.998
Micro-average Precision: 0.998
Micro-average Recall: 0.998
Macro-average Precision: 0.998019801980198
Macro-average Recall: 0.998
Mean Absolute Error (MAE): 0.002
Root Mean Squared Error (RMSE): 0.044721359549995794


# For xgboost test score

random forest test Accuracy : 0.994
test Accuracy : 0.994
Micro-average Precision: 0.994
Micro-average Recall: 0.994
Macro-average Precision: 0.9941747572815534
Macro-average Recall: 0.994
Mean Absolute Error (MAE): 0.012
Root Mean Squared Error (RMSE): 0.15491933384829668
